In [1]:
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.75 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41281,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.75 GiB
Comm: tcp://127.0.0.1:40819,Total threads: 2
Dashboard: http://127.0.0.1:34953/status,Memory: 1.94 GiB
Nanny: tcp://127.0.0.1:33575,


In [33]:
df = dd.read_parquet("trade_view_registration_case_for_prince.parquet").astype("string[pyarrow]").dropna()
df

,countryName,city
npartitions=1,,
,string,string
,...,...


In [34]:
df.to_parquet("result_parquet",partition_on=["countryName"],overwrite=True)

In [35]:
df = dd.read_parquet("result_parquet")
df

,city,countryName
npartitions=227,,
,string,category[known]
,...,...
...,...,...
,...,...
,...,...


In [36]:
df["countryName"].unique()

Dask Series Structure:
npartitions=1
    category[known]
                ...
Name: countryName, dtype: category
Dask Name: unique-agg, 4 graph layers

In [37]:
master = pd.read_parquet("https://github.com/storemesh/masterdata/raw/django-app/master-area_20230810.parquet")
master

,global_code,global_name,region_code,region_name,sub_region_code,sub_region_name,intermediate_region_code,intermediate_region_name,country_or_area,m49_code,iso_alpha2_code,iso_alpha3_code,year,id
0,001,World,142,Asia,034,Southern Asia,000,,Afghanistan,004,AF,AFG,2023,2023AFG
1,001,World,150,Europe,039,Southern Europe,000,,Albania,008,AL,ALB,2023,2023ALB
2,001,World,000,,000,,000,,Antarctica,010,AQ,ATA,2023,2023ATA
3,001,World,002,Africa,015,Northern Africa,000,,Algeria,012,DZ,DZA,2023,2023DZA
4,001,World,009,Oceania,061,Polynesia,000,,American Samoa,016,AS,ASM,2023,2023ASM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,001,World,009,Oceania,061,Polynesia,000,,Samoa,882,WS,WSM,2023,2023WSM
246,001,World,142,Asia,145,Western Asia,000,,Yemen,887,YE,YEM,2023,2023YEM
247,001,World,002,Africa,202,Sub-Saharan Africa,014,Eastern Africa,Zambia,894,ZM,ZMB,2023,2023ZMB
248,001,World,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,001,2023,2023001


In [38]:
master.dropna(subset='iso_alpha2_code',inplace=True)
master_dict = master[["country_or_area","iso_alpha2_code"]].to_dict("records")
master_dict = {elm.get("country_or_area").lower():elm.get("iso_alpha2_code") for elm in master_dict}
master_dict

{'afghanistan': 'AF',
 'albania': 'AL',
 'antarctica': 'AQ',
 'algeria': 'DZ',
 'american samoa': 'AS',
 'andorra': 'AD',
 'angola': 'AO',
 'antigua and barbuda': 'AG',
 'azerbaijan': 'AZ',
 'argentina': 'AR',
 'australia': 'AU',
 'austria': 'AT',
 'bahamas': 'BS',
 'bahrain': 'BH',
 'bangladesh': 'BD',
 'armenia': 'AM',
 'barbados': 'BB',
 'belgium': 'BE',
 'bermuda': 'BM',
 'bhutan': 'BT',
 'bolivia (plurinational state of)': 'BO',
 'bosnia and herzegovina': 'BA',
 'botswana': 'BW',
 'bouvet island': 'BV',
 'brazil': 'BR',
 'belize': 'BZ',
 'british indian ocean territory': 'IO',
 'solomon islands': 'SB',
 'british virgin islands': 'VG',
 'brunei darussalam': 'BN',
 'bulgaria': 'BG',
 'myanmar': 'MM',
 'burundi': 'BI',
 'belarus': 'BY',
 'cambodia': 'KH',
 'cameroon': 'CM',
 'canada': 'CA',
 'cabo verde': 'CV',
 'cayman islands': 'KY',
 'central african republic': 'CF',
 'sri lanka': 'LK',
 'chad': 'TD',
 'chile': 'CL',
 'china': 'CN',
 'christmas island': 'CX',
 'cocos (keeling) isl

In [39]:
class Monad:
    def __init__(self, value):
        self.value = value
        self.status = 'dirty'  # Starting with 'dirty'
        self.message = ''
        self.dtype = object
    def __or__(self, func):
        if self.status == 'dirty': # Only process if 'dirty'
            try:
                x = func(self.value)
                self.value = x
                self.status = 'passed'
                self.message += f'Passed:{func.__name__}()|'
                return self 
            except Exception as e:
                self.message += f'Failed:{func.__name__}():{str(e)}|'
        return self

    def __repr__(self):
        return f'{self.status}({self.value}){self.message}'

    def __str__(self):
        return self.__repr__()

In [40]:
def country_code(x):
    valid_countries = ['TH', 'US', 'UK']
    if x not in valid_countries:
        raise Exception("Invalid code")
    else:
        return x
        
def country_name(x):
    valid_countries ={'Thailand':'TH','United State':'USA', 'United Kingdom':'UK'}
    return valid_countries[x]

def country_name_thai(x):
    valid_countries ={'ไทย':'TH','สหรัฐอเมริกา':'USA', 'อังกฤษ':'UK'}
    return valid_countries[x]

def province_name(x):
    valid_province = province_set
    if x not in valid_province:
        raise Exception("Invalid code")
    else:
        return x

def clean_datetime(x):
    return pd.to_datetime(x,errors="raise")

def country_name_01(x):
    valid_country = master_dict
    return valid_country[x.lower()]

def country_name_02(x):
    master_dict = {"vietnam":"VN",
                    "korea" : "KR",
                   "taiwan": "TW",
                   "united kingdom": "GB",
                   "hong kong" : "HK",
                   "russia" : "RU",
                   "laos" : "LA",
                   "netherlands" : "NL",
                   "turkey" : "TR",
                   "iran" : "IR",
                   "republic of south africa" : "ZA"
                   
                  }
    
    valid_country = master_dict
    return valid_country[x.lower()]

In [41]:
# HONG KONG                      1218
# RUSSIA                         1116
# LAOS                            720
# NETHERLANDS                     654
# TURKEY                          556
# IRAN                            532
# REPUBLIC OF SOUTH AFRICA        414

In [42]:
column_name = 'countryName'
df[column_name+'_result'] = df[column_name].apply(lambda x: Monad(x)|country_name_01 | country_name_02 ,meta=object)
df

/opt/conda/lib/python3.10/site-packages/dask/dataframe/core.py:6984: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


,city,countryName,countryName_result
npartitions=227,,,
,string,category[known],object
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [43]:
result={
    'input':df[column_name],
    'value':df[column_name+'_result'].apply(lambda x: x.value,meta=object),
    'status':df[column_name+'_result'].apply(lambda x: x.status,meta=object),
    'message':df[column_name+'_result'].apply(lambda x: x.message,meta=object),
    }
# df_result['column']=column_name
# df2_result = df_result.explode('message').reset_index(names=['row'])
# df2_result
result

/opt/conda/lib/python3.10/site-packages/dask/dataframe/core.py:6984: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


{'input': Dask Series Structure:
 npartitions=227
     category[known]
                 ...
          ...       
                 ...
                 ...
 Name: countryName, dtype: category
 Dask Name: getitem, 5 graph layers,
 'value': Dask Series Structure:
 npartitions=227
     object
        ...
      ...  
        ...
        ...
 dtype: object
 Dask Name: apply, 6 graph layers,
 'status': Dask Series Structure:
 npartitions=227
     object
        ...
      ...  
        ...
        ...
 dtype: object
 Dask Name: apply, 6 graph layers,
 'message': Dask Series Structure:
 npartitions=227
     object
        ...
      ...  
        ...
        ...
 dtype: object
 Dask Name: apply, 6 graph layers}

In [44]:
ddf_result = dd.concat([v.to_frame(k) for k,v in result.items()], axis=1)
ddf_result

/opt/conda/lib/python3.10/site-packages/dask/dataframe/multi.py:1287: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


,input,value,status,message
npartitions=227,,,,
,category[known],object,object,object
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [45]:
ddf_result.compute()

,input,value,status,message
4861,AFGHANISTAN,AF,passed,Passed:country_name_01()|
5059,AFGHANISTAN,AF,passed,Passed:country_name_01()|
5067,AFGHANISTAN,AF,passed,Passed:country_name_01()|
5794,AFGHANISTAN,AF,passed,Passed:country_name_01()|
6063,AFGHANISTAN,AF,passed,Passed:country_name_01()|
...,...,...,...,...
26769,ZIMBABWE,ZW,passed,Passed:country_name_01()|
32302,ZIMBABWE,ZW,passed,Passed:country_name_01()|
54658,ZIMBABWE,ZW,passed,Passed:country_name_01()|
59126,ZIMBABWE,ZW,passed,Passed:country_name_01()|


In [76]:
df_fail = df2_result[df2_result["status"]=="dirty"]
df_fail

,row,input,value,status,message,column
107,90,PALESTINE,PALESTINE,dirty,Failed:country_name_01():'palestine',countryName
108,90,PALESTINE,PALESTINE,dirty,Failed:country_name_02():'palestine',countryName
191,160,GAMBIA,GAMBIA,dirty,Failed:country_name_01():'gambia ',countryName
192,160,GAMBIA,GAMBIA,dirty,Failed:country_name_02():'gambia ',countryName
240,197,PALESTINE,PALESTINE,dirty,Failed:country_name_01():'palestine',countryName
...,...,...,...,...,...,...
67064,59236,GUINEA ( GUINEE ),GUINEA ( GUINEE ),dirty,Failed:country_name_02():'guinea ( guinee )',countryName
67108,59279,GUINEA ( GUINEE ),GUINEA ( GUINEE ),dirty,Failed:country_name_01():'guinea ( guinee )',countryName
67109,59279,GUINEA ( GUINEE ),GUINEA ( GUINEE ),dirty,Failed:country_name_02():'guinea ( guinee )',countryName
67117,59287,GUINEA ( GUINEE ),GUINEA ( GUINEE ),dirty,Failed:country_name_01():'guinea ( guinee )',countryName


In [77]:
df_fail.groupby("input").size().sort_values(ascending=False)

input
GUINEA  ( GUINEE )             302
BRUNEI                         182
CZECH REPUBLIC                 158
TANZANIA                       122
REUNION                        100
PALESTINE                       72
SYRIA                           66
MACAO                           64
TRINIDAD                        60
GAMBIA                          50
VIRGIN ISLANDS (USA)            38
BOLIVIA                         26
COTE DIVOIRE                    24
EAST TIMOR                      20
ARUBA, BONAIRE AND CURACAO      16
UNITED STATES MISCELLANEO       16
IVORY COAST                     14
BRITISH INDIAN OCEAN TERRI.     14
VENEZUELA                       14
MOLDOVA                         12
SWAZILAND                       12
ST.CHRISTOPHER AND NEVIS        12
HAWAII                          10
TAHITI                          10
SAINT BARTHELEMY                 8
SCOTLAND                         6
UNITED STATES VIRGIN ISL.        6
ZAIRE                            6
SAIPAN        

In [64]:
#df_fail[df_fail["value"]=="VIETNAM"].iloc[0].message

In [71]:
master[master["country_or_area"].str.lower().str.contains("taiwan")]

,global_code,global_name,region_code,region_name,sub_region_code,sub_region_name,intermediate_region_code,intermediate_region_name,country_or_area,m49_code,iso_alpha2_code,iso_alpha3_code,year,id
